In [1]:
import fasttext
import numpy as np
import json
import random
import datetime
import os

Via https://fasttext.cc

```
wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz
wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.it.zip
```

In [4]:
wikimodel = fasttext.FastText.load_model("data/wiki.it/wiki.it.bin")

ccmodel = fasttext.FastText.load_model("data/cc.it.300.bin")

In [5]:
def cosine_distance(first, second, model, decimals=2):
    exact_distance = (np.dot(model[first], model[second])
            /(np.linalg.norm(model[first])*np.linalg.norm(model[second]))
           )
    truncated_distance = np.round(float(exact_distance*100),decimals)
    return truncated_distance

# Parole italiane

```
git clone https://github.com/napolux/paroleitaliane.git
```

In [10]:
parole60  = "data/paroleitaliane/paroleitaliane/60000_parole_italiane.txt"
parole95  = "data/paroleitaliane/paroleitaliane/95000_parole_italiane_con_nomi_propri.txt"

parole280 = "data/paroleitaliane/paroleitaliane/280000_parole_italiane.txt"

with open(parole95, "r") as file:
    words = [w.strip() for w in file.readlines()]

In [11]:
words

['a',
 'ab',
 'abaco',
 'abacuc',
 'abadessa',
 'abalietà',
 'abanese',
 'abano',
 'abarica',
 'abarico',
 'abasia',
 'abate',
 'abatino',
 'abatterica',
 'abatterico',
 'abazia',
 'abbacchiamento',
 'abbacchiando',
 'abbacchiare',
 'abbacchiata',
 'abbacchiato',
 'abbacchiatura',
 'abbacchio',
 'abbacinamento',
 'abbacinando',
 'abbacinare',
 'abbacinata',
 'abbacinato',
 'abbadia',
 'abbadinga',
 'abbadingo',
 'abbado',
 'abbagliamento',
 'abbagliando',
 'abbagliante',
 'abbagliare',
 'abbagliata',
 'abbagliato',
 'abbaglio',
 'abbaiamento',
 'abbaiando',
 'abbaiare',
 'abbaiata',
 'abbaiato',
 'abbaino',
 'abbaio',
 'abballando',
 'abballare',
 'abballata',
 'abballato',
 'abbandonando',
 'abbandonare',
 'abbandonata',
 'abbandonato',
 'abbandono',
 'abbarbagliamento',
 'abbarbagliando',
 'abbarbagliare',
 'abbarbagliata',
 'abbarbagliato',
 'abbarbicamento',
 'abbarbicando',
 'abbarbicare',
 'abbarbicata',
 'abbarbicato',
 'abbarcando',
 'abbarcare',
 'abbarcata',
 'abbarcato',
 'a

In [12]:
with open("data/paroleitaliane/paroleitaliane/1000_parole_italiane_comuni.txt", "r") as file:
    common_words = [w.strip() for w in file.readlines()]
    
common_words = [w for w in common_words if len(w)>2]
len(common_words)

1132

In [16]:
def create_semantle_data(all_words, solution_words, model, date, puzzle_number, verbose=False):
    # Day to use for the solution
    isoday = date.isoformat()
    
    # Solution
    # Seed the random number generator
    random.seed(a=isoday, version=2)
    # Pick the word
    query = random.choice(solution_words)
    if verbose:
        print("La soluzione per il {} è \"{}\"".format(isoday, query))
        
    # Distanze
    distances_list = [(word, cosine_distance(query, word, model)) for word in set(all_words+solution_words) if len(word)>1]
    
    # Distanze nell'ordine giusto
    distances_list_sorted = sorted(distances_list, key=lambda x: x[1],reverse=True)
    
    # Database per Semantle
    
    word_database = {
        _tuple[0]:
        {
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted)
    }
    
    # Le parole più vicine
    closest_words_list = [
        {
            "w": _tuple[0],
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted[:1000])
    ]
    
    solution_word = query
    
    # Statistiche
    
    nearest_word_similarity = closest_words_list[1]["s"]
    tenth_nearest_word_similarity = closest_words_list[9]["s"]
    thousandth_nearest_word_similarity = closest_words_list[999]["s"]

    day_stats = {
        "puzzle_number": puzzle_number,
        "nearest_word_similarity": nearest_word_similarity,
        "tenth_nearest_word_similarity": tenth_nearest_word_similarity,
        "thousandth_nearest_word_similarity": thousandth_nearest_word_similarity,
    }

    semantle_data = {
        "word_database": word_database,
        "closest_words_list": closest_words_list,
        "solution_word": solution_word,
        "day_stats": day_stats,
    }
    return semantle_data

In [17]:
data = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date.today(),
    puzzle_number=1,
    verbose=True
)

La soluzione per il 2022-09-09 è "zio"


In [ ]:
for days_ahead in range(365):
    _date = datetime.date.today()+datetime.timedelta(days=days_ahead)
    data = create_semantle_data(
        all_words=words,
        solution_words=common_words,
        model=wikimodel,
        date=_date,
        puzzle_number=days_ahead,
        verbose=True
    )
    isopath = _date.isoformat()
    os.makedirs(isopath, exist_ok = True)
    with open(isopath+"/semantle.json", "w") as file:
        json.dump(data,file)

La soluzione per il 2022-09-09 è "zio"
La soluzione per il 2022-09-10 è "qualsiasi"
La soluzione per il 2022-09-11 è "nobile"
La soluzione per il 2022-09-12 è "segno"
La soluzione per il 2022-09-13 è "correre"
La soluzione per il 2022-09-14 è "insomma"
La soluzione per il 2022-09-15 è "caratteristico"
La soluzione per il 2022-09-16 è "accompagnare"
La soluzione per il 2022-09-17 è "settimana"
La soluzione per il 2022-09-18 è "rapporto"
La soluzione per il 2022-09-19 è "colore"
La soluzione per il 2022-09-20 è "regione"
La soluzione per il 2022-09-21 è "spesa"
La soluzione per il 2022-09-22 è "realtà"
La soluzione per il 2022-09-23 è "minuto"
La soluzione per il 2022-09-24 è "giornale"
La soluzione per il 2022-09-25 è "entro"
La soluzione per il 2022-09-26 è "necessità"
La soluzione per il 2022-09-27 è "viaggio"
La soluzione per il 2022-09-28 è "tuttavia"
La soluzione per il 2022-09-29 è "capitare"
La soluzione per il 2022-09-30 è "accorgersi"
La soluzione per il 2022-10-01 è "oggi"
La 

La soluzione per il 2023-03-19 è "muovere"
La soluzione per il 2023-03-20 è "mandare"
La soluzione per il 2023-03-21 è "periodo"
La soluzione per il 2023-03-22 è "vita"
La soluzione per il 2023-03-23 è "studiare"
La soluzione per il 2023-03-24 è "strano"
La soluzione per il 2023-03-25 è "leggero"
La soluzione per il 2023-03-26 è "sala"
La soluzione per il 2023-03-27 è "suo"
La soluzione per il 2023-03-28 è "giungere"
La soluzione per il 2023-03-29 è "limitare"
La soluzione per il 2023-03-30 è "mangiare"
La soluzione per il 2023-03-31 è "segreto"
La soluzione per il 2023-04-01 è "presso"
La soluzione per il 2023-04-02 è "figlio"
